In [23]:
#nessicary packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from datetime import datetime
from datetime import date
import time

In [24]:
#get sample data
file = "trip39A.txt"

df = pd.read_csv(file, delimiter = ",")

In [25]:
#drop redundant data
redudantRows = ["DATASOURCE","BASIN","TENDERLOT","SUPPRESSED","JUSTIFICATIONID","LASTUPDATE","NOTE","Unnamed: 0"]

df.drop(columns=redudantRows, inplace=True)
df.shape

(149, 9)

In [26]:
#test target features
df["PLANNEDTIME_TOTAL"] = df["PLANNEDTIME_ARR"] - df["PLANNEDTIME_DEP"]
df["ACTUALTIME_TOTAL"] = df["ACTUALTIME_ARR"] - df["ACTUALTIME_DEP"]
df["DELAY"] = df["ACTUALTIME_TOTAL"] - df["PLANNEDTIME_TOTAL"]

df.shape

(149, 12)

In [27]:
#Drop NA Values
df.dropna(inplace = True)
df.shape

(127, 12)

In [28]:
#format date column correctly
df['DATE'] = np.nan
for i in range(df.shape[0]):
    df['DATE'].iloc[i] = datetime.strptime(df['DAYOFSERVICE'].iloc[i], '%d-%b-%y 00:00:00')
    
#get hour and minutes
df["hours"] = df["ACTUALTIME_DEP"]//3600
df["minutes"] = ((df["ACTUALTIME_DEP"]/3600 - df["ACTUALTIME_DEP"]//3600)*60)//1
df.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


DAYOFSERVICE          object
TRIPID                 int64
LINEID                object
ROUTEID               object
DIRECTION              int64
PLANNEDTIME_ARR        int64
PLANNEDTIME_DEP        int64
ACTUALTIME_ARR       float64
ACTUALTIME_DEP       float64
PLANNEDTIME_TOTAL      int64
ACTUALTIME_TOTAL     float64
DELAY                float64
DATE                  object
hours                float64
minutes              float64
dtype: object

In [30]:
df

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,DATE,hours,minutes
0,07-FEB-18 00:00:00,6246028,39A,39A_40,1,49276,44400,50038.0,44382.0,4876,5656.0,780.0,2018-02-07 00:00:00,12.0,19.0
1,07-FEB-18 00:00:00,6261203,39A,39A_43,2,39535,34800,40249.0,34984.0,4735,5265.0,530.0,2018-02-07 00:00:00,9.0,43.0
2,07-FEB-18 00:00:00,6251013,39A,39A_40,1,71287,66600,72534.0,66602.0,4687,5932.0,1245.0,2018-02-07 00:00:00,18.0,30.0
3,26-JUN-18 00:00:00,7104333,39A,39A_43,2,40074,35400,40759.0,35333.0,4674,5426.0,752.0,2018-06-26 00:00:00,9.0,48.0
4,26-JUN-18 00:00:00,7104336,39A,39A_40,1,58316,53400,58367.0,53393.0,4916,4974.0,58.0,2018-06-26 00:00:00,14.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,18-FEB-18 00:00:00,6264923,39A,39A_43,2,46163,42000,46657.0,42023.0,4163,4634.0,471.0,2018-02-18 00:00:00,11.0,40.0
145,18-FEB-18 00:00:00,6274917,39A,39A_43,2,42863,38700,43234.0,38743.0,4163,4491.0,328.0,2018-02-18 00:00:00,10.0,45.0
146,18-FEB-18 00:00:00,6271772,39A,39A_43,2,41063,36900,41036.0,36945.0,4163,4091.0,-72.0,2018-02-18 00:00:00,10.0,15.0
147,18-FEB-18 00:00:00,6263988,39A,39A_43,2,44663,40500,44979.0,40500.0,4163,4479.0,316.0,2018-02-18 00:00:00,11.0,15.0


In [31]:
#Get weather data
dfWeather = pd.read_csv("2018_historic_weather_3.txt", delimiter = "\t",  parse_dates=False)
dfWeather.dt = dfWeather.dt.astype('datetime64[ns]')
#dfWeather.dt

dfWeather.dtypes

dt                     datetime64[ns]
dt_iso                         object
timezone                        int64
city_name                      object
lat                           float64
lon                           float64
temp                          float64
feels_like                    float64
temp_min                      float64
temp_max                      float64
pressure                        int64
sea_level                     float64
grnd_level                    float64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
rain_1h                       float64
rain_3h                       float64
snow_1h                       float64
snow_3h                       float64
clouds_all                      int64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
dtype: object

In [32]:
#create propper weather columns
dfWeather['dates'] = np.nan

for i in range(8800):
    dfWeather['dates'].iloc[i] = datetime.strptime(dfWeather['dt_iso'].iloc[i], '%Y-%m-%d %H:%M:%S +0000 UTC')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [33]:
#select features that may have proise with tarket outcome
dfWeather["DATE"] = dfWeather['dates']
dfweather1 = dfWeather[["DATE","temp","feels_like","temp_min","temp_max","pressure","humidity","weather_main"]]

In [34]:
dfweather1.DATE = dfweather1.DATE.astype('datetime64[ns]')
dfweather1.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


DATE            datetime64[ns]
temp                   float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure                 int64
humidity                 int64
weather_main            object
dtype: object

In [35]:
#create cleaned wether data set
dfWeather_main = pd.get_dummies(dfweather1['weather_main'])
dfWeather_cleaned = pd.concat([dfweather1[["DATE","temp","feels_like","temp_min","temp_max","pressure","humidity"]],pd.get_dummies(dfweather1['weather_main'])],axis=1)

dfWeather_cleaned["DATES"] = dfWeather_cleaned["DATE"]
dfWeather_cleaned.DATES = dfWeather_cleaned.DATES.astype('datetime64[ns]')
dfWeather_cleaned.dtypes

DATE          datetime64[ns]
temp                 float64
feels_like           float64
temp_min             float64
temp_max             float64
pressure               int64
humidity               int64
Clear                  uint8
Clouds                 uint8
Drizzle                uint8
Fog                    uint8
Mist                   uint8
Rain                   uint8
Smoke                  uint8
Snow                   uint8
DATES         datetime64[ns]
dtype: object

In [65]:
#create datetime column with all the data
df["month"] = np.nan

for i in range(df.shape[0]):
        df["DATE"].iloc[i]= df["DATE"].iloc[i].replace(hour=int(df["hours"].iloc[i]), minute=int(df["minutes"].iloc[i]))
        df["month"].iloc[i] = df.DATE.iloc[i].month

df["month"] = df["month"].astype("int64")        
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,PLANNEDTIME_TOTAL,ACTUALTIME_TOTAL,DELAY,DATE,hours,minutes,month
0,07-FEB-18 00:00:00,6246028,39A,39A_40,1,49276,44400,50038.0,44382.0,4876,5656.0,780.0,2018-02-07 12:19:00,12.0,19.0,2
1,07-FEB-18 00:00:00,6261203,39A,39A_43,2,39535,34800,40249.0,34984.0,4735,5265.0,530.0,2018-02-07 09:43:00,9.0,43.0,2
2,07-FEB-18 00:00:00,6251013,39A,39A_40,1,71287,66600,72534.0,66602.0,4687,5932.0,1245.0,2018-02-07 18:30:00,18.0,30.0,2
3,26-JUN-18 00:00:00,7104333,39A,39A_43,2,40074,35400,40759.0,35333.0,4674,5426.0,752.0,2018-06-26 09:48:00,9.0,48.0,6
4,26-JUN-18 00:00:00,7104336,39A,39A_40,1,58316,53400,58367.0,53393.0,4916,4974.0,58.0,2018-06-26 14:49:00,14.0,49.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,18-FEB-18 00:00:00,6264923,39A,39A_43,2,46163,42000,46657.0,42023.0,4163,4634.0,471.0,2018-02-18 11:40:00,11.0,40.0,2
145,18-FEB-18 00:00:00,6274917,39A,39A_43,2,42863,38700,43234.0,38743.0,4163,4491.0,328.0,2018-02-18 10:45:00,10.0,45.0,2
146,18-FEB-18 00:00:00,6271772,39A,39A_43,2,41063,36900,41036.0,36945.0,4163,4091.0,-72.0,2018-02-18 10:15:00,10.0,15.0,2
147,18-FEB-18 00:00:00,6263988,39A,39A_43,2,44663,40500,44979.0,40500.0,4163,4479.0,316.0,2018-02-18 11:15:00,11.0,15.0,2


In [71]:
#df.DATES.astype('object')
df_final = df[["TRIPID","DATE","DIRECTION","ACTUALTIME_TOTAL","DELAY","LINEID","ROUTEID","hours","month"]]
df_final.DATE = df_final.DATE.astype('datetime64[ns]')
df_final.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


TRIPID                       int64
DATE                datetime64[ns]
DIRECTION                    int64
ACTUALTIME_TOTAL           float64
DELAY                      float64
LINEID                      object
ROUTEID                     object
hours                      float64
month                        int64
dtype: object

In [72]:
df_total = pd.merge_asof(df_final.sort_values("DATE"), dfWeather_cleaned, left_on="DATE", right_on="DATE")

In [73]:
#convert directions to binary column
df_total.DIRECTION.replace(to_replace=2, value=0, inplace=True)

In [75]:
df_total.corr()[["DELAY","ACTUALTIME_TOTAL"]].sort_values(by=['DELAY'], ascending=False)

,DELAY,ACTUALTIME_TOTAL
DELAY,1.000000,0.700393
ACTUALTIME_TOTAL,0.700393,1.000000
DIRECTION,0.263680,0.227464
pressure,0.122984,0.117790
Mist,0.111460,-0.001347
hours,0.051162,-0.151248
Drizzle,0.049660,-0.089720
Clouds,-0.002171,0.262591
temp,-0.043957,0.078778
temp_min,-0.048508,0.056851


In [ ]:
#Direction, 1 = inbound , 0 = outbound

# Notes on potential features
Direction: 26.37% correlation indicates that buses 

In [70]:
df_total

,TRIPID,DATE,DIRECTION,PLANNEDTIME_TOTAL,DELAY,LINEID,ROUTEID,hours,month,temp,...,humidity,Clear,Clouds,Drizzle,Fog,Mist,Rain,Smoke,Snow,DATES
0,6261203,2018-02-07 09:43:00,0,4735,530.0,39A,39A_43,9.0,2,0.09,...,93,0,1,0,0,0,0,0,0,2018-02-07 09:00:00
1,6246028,2018-02-07 12:19:00,1,4876,780.0,39A,39A_40,12.0,2,2.57,...,87,0,1,0,0,0,0,0,0,2018-02-07 12:00:00
2,6251013,2018-02-07 18:30:00,1,4687,1245.0,39A,39A_40,18.0,2,3.47,...,87,0,1,0,0,0,0,0,0,2018-02-07 18:00:00
3,6274915,2018-02-18 08:19:00,0,3571,349.0,39A,39A_43,8.0,2,4.89,...,93,0,1,0,0,0,0,0,0,2018-02-18 08:00:00
4,6271772,2018-02-18 10:15:00,0,4163,-72.0,39A,39A_43,10.0,2,6.63,...,93,0,0,0,0,0,1,0,0,2018-02-18 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,8024541,2018-10-11 09:19:00,0,5190,367.0,39A,39A_43,9.0,10,14.82,...,82,0,1,0,0,0,0,0,0,2018-10-11 09:00:00
123,8025109,2018-10-11 09:29:00,0,5190,397.0,39A,39A_43,9.0,10,14.82,...,82,0,1,0,0,0,0,0,0,2018-10-11 09:00:00
124,8020475,2018-10-11 09:51:00,0,5190,229.0,39A,39A_43,9.0,10,14.82,...,82,0,1,0,0,0,0,0,0,2018-10-11 09:00:00
125,8023269,2018-10-11 18:21:00,0,5121,958.0,39A,39A_43,18.0,10,11.26,...,82,0,1,0,0,0,0,0,0,2018-10-11 18:00:00
